In [3]:
#pip install opencv-python
#pip install os-sys
###
import cv2
import os
import numpy as np
dataPath= "D:\Escuelas\Campus de Robotica\ia\Sin-con-barbijo\Dataset_faces"
dir_list = os.listdir(dataPath)
#print("Lista de archivos: ",dir_list)

labels=[]
facesData =[]
label =0

for name_dir in dir_list:
    dir_path = dataPath + "/" +name_dir
    
    for file_name in os.listdir(dir_path):
        image_path = dir_path +"/" + file_name
        image=cv2.imread(image_path, 0)
        facesData.append(image)
        labels.append(label)
    label+=1
#print("Etiqueta 0: ",np.count_nonzero(np.array(labels)==0))
#print("Etiqueta 1: ",np.count_nonzero(np.array(labels)==1))

#LBPH pip install opencv-contrib-python
face_mask =cv2.face.LBPHFaceRecognizer_create()
#Entrenamiento
face_mask.train(facesData, np.array(labels))
#almacenamiento
face_mask.write("face_mask_model.xml")
print("modelo almacenado")



modelo almacenado


In [14]:
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
LABELS = ["Con_mascarilla","Sin_mascarilla"]
#Leer modelo
face_mask = cv2.face.LBPHFaceRecognizer_create()
face_mask.read("face_mask_model.xml")
cap= cv2.VideoCapture(0, cv2.CAP_DSHOW)
with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
    while True:
        ret,frame=cap.read()
        if ret==False:
            break
        frame=cv2.flip(frame,1)
        
        height,width, _ =frame.shape
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame_rgb)
        if results.detections is not None:
            for detection in results.detections:
                xmin = int(detection.location_data.relative_bounding_box.xmin * width)
                ymin = int(detection.location_data.relative_bounding_box.xmin * height)
                w = int(detection.location_data.relative_bounding_box.width * width)
                h = int(detection.location_data.relative_bounding_box.height * height)
                if xmin < 0 and ymin < 0:
                    continue
                face_image =frame[ymin : ymin + h,xmin :xmin +w]
                face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                face_image = cv2.resize(face_image,(72,72), interpolation=cv2.INTER_CUBIC)

                result = face_mask.predict(face_image)
                #cv2.putText(frame, "{}".format(result), (xmin,ymin -5),1,1.3,(210,124,176),1,cv2.LINE_AA)

                if result[1] < 150:
                    color=(0,255,0) if LABELS[result[0]]=="Con_mascarilla" else (0,0,255)
                    cv2.putText(frame, "{}".format(LABELS[result[0]]), (xmin, ymin -15),2,1,color,1,cv2.LINE_AA)
                    cv2.rectangle(frame, (xmin, ymin), (xmin + w, ymin + h),color,2)

        cv2.imshow("Frame",frame)
        k=cv2.waitKey(1)
        if k==27:
            break
cap.release()
cv2.destroyAllWindows()